In [17]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import json
from PIL import Image
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

SEED=1234
tf.random.set_seed(SEED)

In [18]:
dataset_dir = "../input/anndl-2020-vqa/VQA_Dataset"

imgs_path = os.path.join(dataset_dir, "Images")
train_json_path = os.path.join(dataset_dir, "train_questions_annotations.json")
test_json_path = os.path.join(dataset_dir, "test_questions.json")

In [19]:
# andiamo a definire la divisione train val nei file txt

"""per ogni ogg json prendo obj id (primo elemento) lo metto in una lista
poi prendo la lista e faccio shuffle
creo txt train e val (e li svuoto)
poi prendo i primi 80 % elem della lista e li scrivo nel txt train, il 20 % in txt val"""

# 80:100=x:len x=lenx80:100
import os
import random
import json

list_obj_id = [] 
with open(train_json_path) as data_file:    
    data = json.load(data_file)
data_file.close()
#i = 0
for item in data.items():
    #i = i+1
    obj_value = item[0]
    list_obj_id.append(obj_value)
    #print(obj_value)

random.shuffle(list_obj_id)
lenght = len(list_obj_id)
#print(lenght, i)

lenght_train = 80
lenght_train = int((lenght * lenght_train)/100)

split_dir = os.path.join("./", "Splits")

if not os.path.isdir(split_dir):
    os.mkdir(split_dir)

train_file= open(os.path.join("./", 'Splits/train.txt'),"w+")
train_file.truncate(0)
val_file= open(os.path.join("./", 'Splits/val.txt'),"w+")
val_file.truncate(0)

for i in range (0, lenght_train-1):
    elem = list_obj_id[0]
    train_file.write(elem + "\n")
    list_obj_id.remove(elem)

train_file.close()

lenght = len(list_obj_id)
for i in range (0, lenght):
    elem = list_obj_id[0]
    val_file.write(elem + "\n")
    list_obj_id.remove(elem)

val_file.close()

In [21]:
#class definition
classes = {
    '0': 0,
    '1': 1,
    '2': 2,
    '3': 3,
    '4': 4,
    '5': 5,
    'apple': 6,
    'baseball': 7,
    'bench': 8,
    'bike': 9,
    'bird': 10,
    'black': 11,
    'blanket': 12,
    'blue': 13,
    'bone': 14,
    'book': 15,
    'boy': 16,
    'brown': 17,
    'cat': 18,
    'chair': 19,
    'couch': 20,
    'dog': 21,
    'floor': 22,
    'food': 23,
    'football': 24,
    'girl': 25,
    'grass': 26,
    'gray': 27,
    'green': 28,
    'left': 29,
    'log': 30,
    'man': 31,
    'monkey bars': 32,
    'no': 33,
    'nothing': 34,
    'orange': 35,
    'pie': 36,
    'plant': 37,
    'playing': 38,
    'red': 39,
    'right': 40,
    'rug': 41,
    'sandbox': 42,
    'sitting': 43,
    'sleeping': 44,
    'soccer': 45,
    'squirrel': 46,
    'standing': 47,
    'stool': 48,
    'sunny': 49,
    'table': 50,
    'tree': 51,
    'watermelon': 52,
    'white': 53,
    'wine': 54,
    'woman': 55,
    'yellow': 56,
    'yes': 57
}

N_CLASSES = len(classes)

# Preprocessing

Here we do some preprocessing on the questions/answers sentences using a Tokenizer to encode them.

In [22]:
with open(train_json_path) as data_file:    
    data = json.load(data_file)         
data_file.close()

with open(test_json_path) as dataTest_file:    
    data_test = json.load(dataTest_file)         
dataTest_file.close()

questions_sentences = [] # input encoder 
answers_sentences = [] # target sentences
    
for item in data.items():
    question = item[1]["question"]
    question = question.replace("?", "")
    answer = item[1]["answer"]
    questions_sentences.append(question)
    
for item_test in data_test.items():
    question = item[1]["question"]
    question = question.replace("?", "")
    answer = item[1]["answer"]
    questions_sentences.append(question)

# QUESTIONS PAD
question_tokenizer = Tokenizer()
question_tokenizer.fit_on_texts(questions_sentences)
question_tokenized = question_tokenizer.texts_to_sequences(questions_sentences)
question_wtoi = question_tokenizer.word_index
max_question_length = max(len(sentence) for sentence in question_tokenized)

In [23]:
from PIL import Image

class CustomDataset_(tf.keras.utils.Sequence):
    def __init__(self, dataset_dir, which_subset, json_file, classes, n_classes, 
                 question_tokenizer, max_question_length):
       
        if which_subset == 'training':
            subset_file = os.path.join('Splits', 'train.txt')
        elif which_subset == 'validation':
            subset_file = os.path.join('Splits', 'val.txt')
        
        with open(subset_file, 'r') as f:
            lines = f.readlines()
            
        subset_filenames = []
        
        for line in lines:
            subset_filenames.append(line.strip()) 
            self.which_subset = which_subset
            self.dataset_dir = dataset_dir 
            self.subset_filenames = subset_filenames

        with open(json_file) as data_file:    
            data = json.load(data_file)         
        data_file.close()
        
        data_copy=data.copy()
        data_sub={k: data_copy[k] for k in subset_filenames}
        
        for item in data_sub.items():
            question = item[1]["question"]
            question = question.replace("?", "")
            answer = item[1]["answer"]
            questions_sentences.append(question)
            answers_sentences.append(answer)
        
        question_tokenized = question_tokenizer.texts_to_sequences(questions_sentences)
        question_wtoi = question_tokenizer.word_index
        question_padded = pad_sequences(question_tokenized, maxlen=max_question_length)
      
        for e,item in enumerate(data_sub.items()):
            data_sub[item[0]]['question']=question_padded[e]
            data_sub[item[0]]['answer']=answers_sentences[e]
        
        self.classes = classes
        self.n_classes = n_classes
        self.data = data
        self.data_sub=data_sub
        self.question_tokenized = question_tokenized
        self.max_question_length = max_question_length
        self.question_tokenizer=question_tokenizer

    def __len__(self):
        return len(self.subset_filenames)

    def __getitem__(self, index):
        curr_filename = self.subset_filenames[index]
        obj = self.data_sub[curr_filename]

        image_id = obj['image_id']
        image_name = image_id + ".png"
        img = Image.open(os.path.join(self.dataset_dir, image_name))
        img = img.convert('RGB')
        img_arr = np.asarray(img)
        img_arr=img_arr[None,...]

        question_tokenized = obj['question']
        question_tokenized = question_tokenized[None,...]
         
        answer = obj['answer']
        answer_num = self.classes[answer]
        n= np.zeros(1)
        n[0] = answer_num
        answer = n[None,...]

        return (np.float32(question_tokenized), np.float32(img_arr)), np.float32(answer) 


In CustomDataset_ we generate our tuples of question-image and answer to train our network. We have used a dictionary with question, image_id and answer as key and the question_encoded, the image_id and the answer_label as values.

# Dataset Creation

We create our training set and validation set similarly as we have done in the segmentation challenge, using the CustomDataset_ class.

In [24]:

dataset_dir = "../input/anndl-2020-vqa/VQA_Dataset"


img_h = 400
img_w = 700
num_classes=58

dataset = CustomDataset_(imgs_path, 'training', train_json_path, classes, num_classes, 
                         question_tokenizer, max_question_length)

dataset_valid = CustomDataset_(imgs_path, 'validation', train_json_path, classes, num_classes,
                            question_tokenizer, max_question_length)



In [25]:
train_dataset = tf.data.Dataset.from_generator(lambda: dataset,
                                               output_types=((tf.int32, tf.float32), tf.float32),
                                               output_shapes=(([None, dataset.max_question_length], [None, img_h, img_w, 3]), [None, 1]))#None,img_h, img_w, 3

train_dataset = train_dataset.repeat()

valid_dataset = tf.data.Dataset.from_generator(lambda: dataset_valid,
                                               output_types=((tf.int32, tf.float32), tf.float32),
                                               output_shapes=(([None, dataset_valid.max_question_length], [None, img_h, img_w, 3]), [None, 1]))

valid_dataset = valid_dataset.repeat()

train_dataset


MAX_LENGTH=dataset.max_question_length
print(MAX_LENGTH)

words_number=(len(question_wtoi)+1)

21


# Model(s)

In the vqa_model_1 we uses the fine-tuning technique, in particular we use the VGG as pretrained model and a stack of two bidirectional lstm as RNN (as in the model with the custom CNN). While in vqa_model_2 we use Inception and a Gap layer at the end (for the CNN part).

In [26]:

import numpy as np
import json
import cv2
from keras import backend as K

import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences

def vqa_model_1(img_h,img_w,MAX_LENGTH,words_number):
    base_model = tf.keras.applications.VGG16(input_shape=(img_h, img_w, 3), include_top=False, weights='imagenet')
    finetuning = True

    if finetuning:
        freeze_until = 15 # layer from which we want to fine-tune

        for layer in base_model.layers[:freeze_until]:
            layer.trainable = False

    vision_model = tf.keras.models.Sequential()

    vision_model.add(base_model)

    vision_model.add(tf.keras.layers.Dropout(0.1))

    vision_model.add(tf.keras.layers.Dense(1024))
    vision_model.add(tf.keras.layers.Flatten())
    image_input = tf.keras.layers.Input(shape=(img_h, img_w, 3))
    encoded_image = vision_model(image_input)

    # Define RNN for language input
    question_input = tf.keras.layers.Input(shape=[MAX_LENGTH])
    embedded_question = tf.keras.layers.Embedding(input_dim=words_number, output_dim=512, input_length=MAX_LENGTH)(question_input)
    encoded_question = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(200, dropout=0.3, return_sequences=True, recurrent_dropout=0.1, unroll=True))(embedded_question)
    encoded_question = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(200, dropout=0.1, recurrent_dropout=0.1, unroll=True))(encoded_question)
    # Combine CNN and RNN to create the final model
    merged = tf.keras.layers.concatenate([encoded_question, encoded_image])
    output = tf.keras.layers.Dense(128)(merged)
    output = tf.keras.layers.Dropout(0.1)(output)
    output = tf.keras.layers.Dense(N_CLASSES, activation='softmax')(output)
    vqa_model_1 = tf.keras.models.Model(inputs=[question_input, image_input], outputs=output)
    
    vqa_model_1.summary()
    return vqa_model_1

In [27]:
EMBEDDING_SIZE = 128


from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dropout, Input, Flatten, Dense
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import concatenate
from tensorflow.keras.applications.inception_v3 import InceptionV3

def vqa_model_2(img_w, img_h,max_length,q_wtoi):
    # CNN
    inputs_cnn = Input((img_w, img_h, 3))    
    inc = InceptionV3(include_top=False, weights='imagenet',input_shape=(400, 700, 3))(inputs_cnn)
    gap = GlobalAveragePooling2D()(inc)
        
    # LSTM
    encoder_input = tf.keras.Input(shape=[max_length])
    embedding = tf.keras.layers.Embedding(q_wtoi, EMBEDDING_SIZE, input_length=max_length) (encoder_input)
    encoded_question = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(200, dropout=0.3, return_sequences=True, recurrent_dropout=0.1, unroll=True))(embedding)
    encoded_question = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(200, dropout=0.1, recurrent_dropout=0.1, unroll=True))(encoded_question)
    
    concat_layer = concatenate([gap, encoded_question])
    outputs = Dense(units=58, activation='softmax') (concat_layer)
    
    model = Model(inputs=[encoder_input,inputs_cnn], outputs=[outputs])
    return model

Set VQA_1 to true if you want to train vqa_model_1, to false if you want to train vqa_model_2.

In [28]:
VQA_1=False

if VQA_1:
    model=vqa_model_1(400, 700,MAX_LENGTH,words_number)
else:
    model=vqa_model_2(400, 700,MAX_LENGTH,words_number)

In [29]:
# Optimization params
# -------------------

# Loss
loss =tf.keras.losses.SparseCategoricalCrossentropy()
# learning rate
lr = 1e-4
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
# -------------------

# Validation metrics
# ------------------

metrics = ['accuracy']
# ------------------

# Compile Model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [30]:
import os
from datetime import datetime

bs=32

cwd = os.getcwd()

exps_dir = os.path.join(cwd, 'vqa_exps')
if not os.path.exists(exps_dir):
    os.makedirs(exps_dir)

now = datetime.now().strftime('%b%d_%H-%M-%S')

exp_name = 'exp'

exp_dir = os.path.join(exps_dir, exp_name + '_' + str(now))
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)
    
callbacks = []

# Model checkpoint
# ----------------
ckpt_dir = os.path.join(exp_dir, 'ckpts')
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp_{epoch:02d}.ckpt'), 
                                                   save_weights_only=True)  # False to save the model directly
callbacks.append(ckpt_callback)

# ----------------

# Visualize Learning on Tensorboard
# ---------------------------------
tb_dir = os.path.join(exp_dir, 'tb_logs')
if not os.path.exists(tb_dir):
    os.makedirs(tb_dir)
    
# By default shows losses and metrics for both training and validation
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir,
                                             profile_batch=0,
                                             histogram_freq=1)  # if 1 shows weights histograms
callbacks.append(tb_callback)

# Early Stopping
# --------------
early_stop = False
if early_stop:
    es_callback = tf.keras.callback.EarlyStopping(monitor='val_loss', patience=8)
    callbacks.append(es_callback)


In [ ]:
model.fit(train_dataset,
          epochs=50,
          steps_per_epoch=len(dataset)//32,
          validation_data=valid_dataset,
          validation_steps=len(dataset_valid)// 32, 
          #validation_split=0.2, 
          callbacks=callbacks)

Epoch 1/50
 123/1470 [=>............................] - ETA: 5:35 - loss: 3.4805 - accuracy: 0.1242

In [ ]:
model.load_weights('/kaggle/working/vqa_exps/exp_Jan31_09-53-13/ckpts/cp_40.ckpt')

# Evaluation

In [ ]:
with open(test_json_path) as data_file:    
    data_test_final = json.load(data_file)
data_file.close()

csv_fname = 'results_'
csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'
with open(os.path.join('./', csv_fname), 'w') as f:
    f.write('Id,Category\n')
    for item in data_test_final.items():
        obj_value = item[0] #id
   
        image_id = item[1]['image_id']
        image_name = image_id + ".png"
        img = Image.open(os.path.join(imgs_path, image_name))
        img = img.convert('RGB')
        img_arr = np.asarray(img)
        img_arr=img_arr[None,...]

        question = item[1]['question']
        question_list = []
        question_list.append(question)
        question_tokenized = question_tokenizer.texts_to_sequences(question_list)
        question_padded = pad_sequences(question_tokenized, maxlen=max_question_length)
        question_tokenized = question_padded[0]
        question_tokenized = question_tokenized[None,...]
        
        input_net = [np.float32(question_tokenized), np.float32(img_arr)]
        prediction = np.argmax(model.predict(input_net))
        f.write(obj_value + ',' + str(prediction) + '\n')
        
print('FINE')
        